### Install

In [1]:
#pip install ucimlrepo

### Imports

In [2]:
# Imports
from ucimlrepo import fetch_ucirepo
import pandas as pd
import psycopg2
from collections import defaultdict
from scipy import stats as kl
from decimal import Decimal
import numpy as np
from scipy.stats import entropy, wasserstein_distance
from itertools import product
import matplotlib.pyplot as plt
from itertools import product


In [3]:
connection = psycopg2.connect(database="Project645", user="vaishnavishah", password="10jan2001", host="localhost", port=5432)
#connection = psycopg2.connect(database="Project645", user="postgres", password="Rajada1234@", host="localhost", port=5433)
#connection = psycopg2.connect(database="seedb", user="postgres", password="nineten910", host="localhost", port=5432)
cursor = connection.cursor()


In [4]:
# checking execution
query = "select workclass from married limit 5;"

cursor.execute(query)
record = cursor.fetchall()
print(record)

[('Self-emp-not-inc',), ('Private',), ('Private',), ('Private',), ('Private',)]


In [5]:
aggregation_functions = ['avg', 'sum', 'max', 'min', 'count']
group_by_columns = [
    'workclass',
    'education',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native_country',
    'income'
]
measure_columns = [
    'age',
    'fnlwgt',
    'education_num',
    'capital_gain',
    'capital_loss',
    'hours_per_week'
]

In [6]:
# FORMULAS 

min_fl = np.finfo(float).eps
def kl_distance(target, reference):
    # print('Target ',target)
    # print('Ref ',reference)
    target = dict(target)
    reference = dict(reference)

    t = list()
    r = list()

    # we need this because this ensures that there are the same number of categories in each distribution
    for key in set().union(target.keys(), reference.keys()):
        t_val = float(target.get(key, 0))
        r_val = float(reference.get(key, 0))
        t.append(t_val)
        r.append(r_val)


    # normalize to create a probability distribution
    t = np.asarray(t) / (np.sum(t) or 1.0)
    r = np.asarray(r) / (np.sum(r) or 1.0)
    t = [max(x, min_fl) for x in t]
    r = [max(x, min_fl) for x in r]


    return entropy(t, r)

def hoeffding_serfling_interval(phase, num_phases, delta):
    phase = float(phase)
    num_phases = float(phase)
    numerator = (1 - ((phase - 1) / num_phases)) * (2 * np.log(np.log(phase)) + np.log((np.pi * np.pi) / (3 * delta)))
    epsilon_m = np.sqrt(numerator / (2 * phase))
    return epsilon_m

In [7]:
# age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country

In [8]:
def naive_search(cur, views_ls, top_k=5, verbose=False):

    grouped_views = defaultdict(list)

    for view in views_ls:
        grouper = view[0]
        grouped_views[grouper].append(view)

    query = "select {group}, {func}({m_col}) from {table} {where} group by {group};"

    results = list()
    for group, views in grouped_views.items():
        for view in views:
            _, func, m_col = view
            target_query = query.format(group=group,
                                        func=func,
                                        m_col=m_col,
                                        table='married',
                                        where=""
                                        )

            cur.execute(target_query)
            target_results = cur.fetchall()

            reference_query = query.format(group=group,
                                           func=func,
                                           m_col=m_col,
                                           table='unmarried',
                                           where=""
                                           )
            cur.execute(reference_query)
            reference_results = cur.fetchall()

            dist = kl_distance(target_results, reference_results)
            results.append((view, dist))

            if verbose:
                print ("{}, {}, {}".format(group, func, m_col), dist)

    return list(reversed(sorted(results, key=lambda x: x[1])))[:top_k]


In [9]:
def get_top_k(cur, grouped_views, top_k=5, verbose=False):
    query_template = "select {group} {aggregated_measures} from {table} {where} group by {group};"

    results = []
    for group, pairs in grouped_views.items():

        aggregated_measures = "".join(", {func}({col})".format(func=func, col=col) for _, func, col in pairs)
        target_query = query_template.format(group=group,
                                    aggregated_measures=aggregated_measures,
                                    table='married',
                                    where=""
                                    )
        cur.execute(target_query)
        t_v = cur.fetchall()

        reference_query = query_template.format(group=group,
                                       aggregated_measures=aggregated_measures,
                                       table='unmarried',
                                       where="")
        cur.execute(reference_query)
        r_v = cur.fetchall()

        for i, view in enumerate(pairs):
            target_results = [(v[0], v[i+1]) for v in t_v]
            reference_results = [(v[0], v[i+1]) for v in r_v]

            dist = kl_distance(target_results, reference_results)
            if verbose:
                print( view, dist)
            results.append((view, dist))

    return list(reversed(sorted(results, key=lambda x: x[1])))[:top_k]

In [10]:
def sharing_based_search(cur, views_ls, top_k=5, verbose=False):
    
    query_template = "select {group} {aggregated_measures} from {table} {where} group by {group};"

    results = []
    grouped_views = defaultdict(list)

    for view in views_ls:
        grouper = view[0]
        grouped_views[grouper].append(view)

    return get_top_k(cur, grouped_views, top_k, verbose)

In [11]:
def pruning_based_search(cur, views, top_k=5, verbose=False):
    mean_utils = {}

    for view, utility in naive_search(cur, views, top_k=None):
        mean_utils[view] = (mean_utils.get(view, 0) + utility) / 2

    sorted_utils = sorted(mean_utils.values())
    max_utility = sorted_utils[-1]
    kth_utility = sorted_utils[-top_k] / max_utility

    for i in range(len(views)):
        if i == 0:
            if verbose:
                print("No pruning on the first iteration due to NaN epsilon.")
            continue

        epsilon = hoeffding_serfling_interval(i + 1, len(views), 0.1) 

        not_pruned_views = [v for v in views if (mean_utils[v] / max_utility + epsilon) >= (kth_utility - epsilon)]
        if verbose:
            print("Iteration %d: Pruned %d views." % (i, len(views) - len(not_pruned_views)))

        views = not_pruned_views[:]

    if verbose:
        print("Finished with %d views." % len(views))

    return views


In [12]:
views_ls = []
for a in group_by_columns:
    for f in aggregation_functions:
        for m in measure_columns:
            views_ls.append((a,f,m))


for i in range(1):
    top_5_prun = pruning_based_search(cursor, views_ls,verbose=True)

Target  [('Without-pay', Decimal('57.2142857142857143')), ('Self-emp-not-inc', Decimal('47.3603428126217374')), ('State-gov', Decimal('43.4854910714285714')), ('Private', Decimal('41.3204585778626996')), ('Self-emp-inc', Decimal('47.6477272727272727')), ('Federal-gov', Decimal('44.8841379310344828')), ('Local-gov', Decimal('44.0077369439071567'))]
Ref  [('Without-pay', Decimal('21.1666666666666667')), ('Self-emp-not-inc', Decimal('38.5029411764705882')), ('State-gov', Decimal('34.8117021276595745')), ('Private', Decimal('31.7769180013196569')), ('Self-emp-inc', Decimal('37.4571428571428571')), ('Federal-gov', Decimal('38.4285714285714286')), ('Local-gov', Decimal('36.9664889565879665'))]
Target  [('Without-pay', Decimal('178319.428571428571')), ('Self-emp-not-inc', Decimal('173504.530580444098')), ('State-gov', Decimal('182122.382812500000')), ('Private', Decimal('190317.178586051590')), ('Self-emp-inc', Decimal('178394.691558441558')), ('Federal-gov', Decimal('183714.074482758621')), 

In [13]:
for i in range(1):

    top_5_shar = sharing_based_search(cursor, views_ls, verbose=False)

Target  [('Without-pay', Decimal('57.2142857142857143')), ('Self-emp-not-inc', Decimal('47.3603428126217374')), ('State-gov', Decimal('43.4854910714285714')), ('Private', Decimal('41.3204585778626996')), ('Self-emp-inc', Decimal('47.6477272727272727')), ('Federal-gov', Decimal('44.8841379310344828')), ('Local-gov', Decimal('44.0077369439071567'))]
Ref  [('Without-pay', Decimal('21.1666666666666667')), ('Self-emp-not-inc', Decimal('38.5029411764705882')), ('State-gov', Decimal('34.8117021276595745')), ('Private', Decimal('31.7769180013196569')), ('Self-emp-inc', Decimal('37.4571428571428571')), ('Federal-gov', Decimal('38.4285714285714286')), ('Local-gov', Decimal('36.9664889565879665'))]
Target  [('Without-pay', Decimal('178319.428571428571')), ('Self-emp-not-inc', Decimal('173504.530580444098')), ('State-gov', Decimal('182122.382812500000')), ('Private', Decimal('190317.178586051590')), ('Self-emp-inc', Decimal('178394.691558441558')), ('Federal-gov', Decimal('183714.074482758621')), 

In [14]:
top_5_shar

[(('relationship', 'sum', 'capital_gain'), 35.15956612486812),
 (('relationship', 'sum', 'capital_loss'), 34.87747673512298),
 (('relationship', 'sum', 'age'), 34.296064194395626),
 (('relationship', 'sum', 'hours_per_week'), 34.276281101060064),
 (('relationship', 'sum', 'education_num'), 34.264130484222676)]

In [15]:
top_5_prun

[('relationship', 'sum', 'age'),
 ('relationship', 'sum', 'fnlwgt'),
 ('relationship', 'sum', 'education_num'),
 ('relationship', 'sum', 'capital_gain'),
 ('relationship', 'sum', 'capital_loss'),
 ('relationship', 'sum', 'hours_per_week'),
 ('relationship', 'count', 'age'),
 ('relationship', 'count', 'fnlwgt'),
 ('relationship', 'count', 'education_num'),
 ('relationship', 'count', 'capital_gain'),
 ('relationship', 'count', 'capital_loss'),
 ('relationship', 'count', 'hours_per_week')]

In [16]:
if __name__ == "__main__":

   
    views_ls = []
    for a in group_by_columns:
        for f in aggregation_functions:
            for m in measure_columns:
                views_ls.append((a,f,m))


    for i in range(1):
        top_5 = pruning_based_search(cursor, views_ls,verbose=True)

        #top_5 = sharing_based_search(cursor, views_ls, verbose=False)

    for view, utility in top_5:
        print (view, utility)

        cursor.execute("select {g}, {f}({m}) from married group by {g};".format(g=view[0], f=view[1], m=view[2]))
        target = dict(cursor.fetchall())

        cursor.execute("select {g}, {f}({m}) from unmarried group by {g};".format(g=view[0], f=view[1], m=view[2]))
        reference = dict(cursor.fetchall())

        t = list()
        r = list()
        names = list()

        for key in set().union(target.keys(), reference.keys()):
            names.append(key)
            t_val = float(target.get(key, 0))
            r_val = float(reference.get(key, 0))
            t.append(t_val)
            r.append(r_val)
        
        width=0.35
        index = np.arange(len(names))

        fig, ax = plt.subplots(figsize=(10, 6)) 
        
        light_pink = '#FFB6C1'  
        pastel_blue = '#AEC6CF' 

        t_bar = ax.bar(index, t, width, color=light_pink)
        r_bar = ax.bar(index + width, r, width, color=pastel_blue)
        
        ax.set_title("{f}({m}) Grouped by {g} (Measure = {measure}, Utility = {util})".format(f=view[1],
        m=view[2],
        g=view[0],
        measure="KLD",
        util=utility),
        fontsize=16, color='navy')  
        ax.set_ylabel("{f}({m})".format(f=view[1], m=view[2]), fontsize=14, color='purple')  
        ax.set_xlabel("Categories of {g}".format(g=view[0]), fontsize=14, color='green') 
        ax.set_xticks(index + width / 2)
        ax.set_xticklabels(names, fontsize=12, color='gray', rotation=45)  

        ax.legend((t_bar[0], r_bar[0]), ('Married', 'Unmarried'), loc='upper right', fontsize=12)

        ax.grid(True, linestyle='--', linewidth=0.5, color='lightgray')  

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.tight_layout()  
        plt.show()

    connection.commit() 

Target  [('Without-pay', Decimal('57.2142857142857143')), ('Self-emp-not-inc', Decimal('47.3603428126217374')), ('State-gov', Decimal('43.4854910714285714')), ('Private', Decimal('41.3204585778626996')), ('Self-emp-inc', Decimal('47.6477272727272727')), ('Federal-gov', Decimal('44.8841379310344828')), ('Local-gov', Decimal('44.0077369439071567'))]
Ref  [('Without-pay', Decimal('21.1666666666666667')), ('Self-emp-not-inc', Decimal('38.5029411764705882')), ('State-gov', Decimal('34.8117021276595745')), ('Private', Decimal('31.7769180013196569')), ('Self-emp-inc', Decimal('37.4571428571428571')), ('Federal-gov', Decimal('38.4285714285714286')), ('Local-gov', Decimal('36.9664889565879665'))]
Target  [('Without-pay', Decimal('178319.428571428571')), ('Self-emp-not-inc', Decimal('173504.530580444098')), ('State-gov', Decimal('182122.382812500000')), ('Private', Decimal('190317.178586051590')), ('Self-emp-inc', Decimal('178394.691558441558')), ('Federal-gov', Decimal('183714.074482758621')), 

ValueError: too many values to unpack (expected 2)

In [ ]:
cursor.close()
connection.close()

In [ ]:
# age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country